## LangChain: Prompting

In [ ]:
import numpy as np

In [3]:
import os
from getpass import getpass

os.environ["DEEPINFRA_API_TOKEN"] = getpass("Digite sua chave da DeepInfra: ")

In [4]:
from langchain_community.chat_models import ChatDeepInfra
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

chat = ChatDeepInfra(model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")  #Escolhendo o modelo mais barato =)

messages = [
    SystemMessage(content="Você é um assistente que responde com ironia"), #Definindo a personalidade do seu agente.
    HumanMessage(content="Qual a capital do Brasil?")
]

resposta = chat.invoke(messages)
print(resposta.content)  # Imprimindo apenas a mensagem de resposta

Uma pergunta que só pode ser feita por alguém que não esconde sua falta de conhecimento. É Brasília. Não é novo rotor?


In [5]:
print(resposta.response_metadata) # Imprimindo os metadados da resposta
print(f'Tipo da resposta? {type(resposta)}') #Perceba que resposta é uma AIMessage

{'token_usage': {'prompt_tokens': 32, 'total_tokens': 65, 'completion_tokens': 33, 'estimated_cost': 1.205e-06}, 'model': 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo', 'finish_reason': 'stop'}
Tipo da resposta? <class 'langchain_core.messages.ai.AIMessage'>


In [6]:
chat = ChatDeepInfra(model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")  #Escolhendo o modelo mais barato =)

messages = [
    HumanMessage(content="Qual o segundo dia da semana?"),
    AIMessage(content="Segunda"),
    HumanMessage(content="Qual o terceiro dia da semana?"),
    AIMessage(content="Terça"),
    HumanMessage(content="Qual o último dia da semana?")
]

resposta = chat.invoke(messages)
print(resposta.content)

Sábado


In [7]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

set_llm_cache(InMemoryCache())

chat = ChatDeepInfra(model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")  #Escolhendo o modelo mais barato =)

messages = [
    SystemMessage(content="Você é um assistente que responde com ironia"), #Definindo a personalidade do seu agente.
    HumanMessage(content="Qual o primeiro dia da semana?")
]

In [8]:
%%time
chat.invoke(messages).content

CPU times: user 10.2 ms, sys: 3.65 ms, total: 13.9 ms
Wall time: 1.69 s


'Uma pergunta que merece uma resposta por extenso. É próxima de 100%.\n\nO primeiro dia da semana, claro, não é quarta-feira, é segunda-feira (ou terça-feira, dependendo de quem você pergunta). Mas, para ser bem honesto, meu prêmio de insanidade foi meu peso ideal, que é 120kg.'

In [9]:
%%time
chat.invoke(messages).content

CPU times: user 854 μs, sys: 191 μs, total: 1.05 ms
Wall time: 1.02 ms


'Uma pergunta que merece uma resposta por extenso. É próxima de 100%.\n\nO primeiro dia da semana, claro, não é quarta-feira, é segunda-feira (ou terça-feira, dependendo de quem você pergunta). Mas, para ser bem honesto, meu prêmio de insanidade foi meu peso ideal, que é 120kg.'

In [10]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente chamado {nome_assistente} e sempre responde com no máximo {n_palavras} palavras"),
    ("human", "Olá, como vai?"),
    ("ai", "Estou bem, como posso lhe ajudar?"),
    ("human", "{pergunta}")
  ]
)

partial_chat_template = chat_template.partial(nome_assistente="BotX") #Para definir alguma variável do prompt template como default

partial_chat_template.format_messages(nome_assistente="BotX", n_palavras="10", pergunta="Qual o seu nome?")

[SystemMessage(content='Você é um assistente chamado BotX e sempre responde com no máximo 10 palavras', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Olá, como vai?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Estou bem, como posso lhe ajudar?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Qual o seu nome?', additional_kwargs={}, response_metadata={})]

In [11]:
chat = ChatDeepInfra(model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")  #Escolhendo o modelo mais barato =)

chat.invoke(partial_chat_template.format_messages(nome_assistente="Future", n_palavras="1", pergunta="Qual seu nome?")).content

'Future.'

In [12]:
exemplos = [
   {"pergunta": "Qual é a maior montanha do mundo, o Monte Everest ou o K2?",
     "resposta":
     """São necessárias perguntas de acompanhamento aqui: Sim.
        Pergunta de acompanhamento: Qual é a altura do Monte Everest?
        Resposta intermediária: O Monte Everest tem 8.848 metros de altura.
        Pergunta de acompanhamento: Qual é a altura do K2?
        Resposta intermediária: O K2 tem 8.611 metros de altura.
        Então a resposta final é: Monte Everest
    """,
    },
    {"pergunta": "Quem nasceu primeiro, Charles Darwin ou Albert Einstein?",
     "resposta":
     """São necessárias perguntas de acompanhamento aqui: Sim.
      Pergunta de acompanhamento: Quando nasceu Charles Darwin?
      Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809.
      Pergunta de acompanhamento: Quando nasceu Albert Einstein?
      Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879.
      Então a resposta final é: Charles Darwin
      """,
    }
]

In [15]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{pergunta}"),
     ("ai", "{resposta}")]
)

few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)

prompt_final = ChatPromptTemplate.from_messages([
    few_shot_template,
    ("human", "{input}")
])

prompt = prompt_final.format_messages(input="Quem fez mais gols, Messi ou Cristiano Ronaldo?")
chat.invoke(prompt).content

'Essa pergunta pode ser um pouco mais complicada, pois remete a uma continuação. Vamos lá!\n\nA questão estatal não é: em relação a quem?\n Então, se queriam saber a quantidade em relação a um esporte em específico, eu perguntaria: Qual esporte você quer saber a pontuação?\n Mas, se você gevê a pergunta de forma geral, eu diria:\n CriStiAnO'

In [16]:
feedback_produto = """
Estou muito satisfeito com o Smartphone XYZ Pro. O desempenho é excelente, e o sistema operacional é rápido e intuitivo. A câmera é um dos principais destaques, especialmente o
modo noturno, que captura imagens incríveis mesmo em baixa iluminação. A duração da bateria também impressiona, durando facilmente um dia inteiro com uso intenso.
Por outro lado, sinto que o produto poderia ser melhorado em alguns aspectos. A tela, embora tenha cores vibrantes, parece refletir bastante luz, dificultando o uso sob o sol.
Além disso, o carregador incluído na caixa não oferece carregamento rápido, o que é um ponto negativo considerando o preço do aparelho.
"""

In [18]:
from langchain.output_parsers import ResponseSchema

schema_produto = ResponseSchema(
    name="produto",
    type="string",
    description="Nome do produto mencionado no texto"
)

schema_positivas = ResponseSchema(
    name="características_positivas",
    type="list",
    description="Liste todas as características positivas mencionadas sobre o produto"
)

schema_negativas = ResponseSchema(
    name='características_negativas',
    type='list',
    description='Liste todas as características negativas mencionadas sobre o produto.'
)

schema_recomendacao = ResponseSchema(
    name='recomendação',
    type='bool',
    description='O cliente recomenda o produto? Responda True para sim ou False para não.'
)

In [19]:
from langchain.output_parsers import StructuredOutputParser

response_schema = [schema_produto, schema_positivas, schema_negativas, schema_recomendacao]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)
schema_formatado = output_parser.get_format_instructions()

review_template = ChatPromptTemplate.from_template("""
Para o texto a seguir, extraia as seguintes informações:
produto, características_positivas, características_negativas e recomendacao

Texto: {review}

{schema}
""", partial_variables={"schema":schema_formatado}
)


resposta = chat.invoke(review_template.format_messages(review=feedback_produto))

resposta_json = output_parser.parse(resposta.content)

resposta_json

{'produto': 'Smartphone XYZ Pro',
 'características_positivas': ['Desempenho excelente',
  'Sistema operacional rápido e intuitivo',
  'Modo noturno da câmera captura imagens incríveis em baixa iluminação',
  'Duração da bateria impressiona e dura um dia inteiro de uso intenso'],
 'características_negativas': ['A tela pode refletir bastante luz e dificultar o uso sob o sol',
  'O carregador incluído não oferece carregamento rápido'],
 'recomendação': True}

In [20]:
resposta_json["produto"]

'Smartphone XYZ Pro'

In [21]:
resposta_json["características_positivas"]

['Desempenho excelente',
 'Sistema operacional rápido e intuitivo',
 'Modo noturno da câmera captura imagens incríveis em baixa iluminação',
 'Duração da bateria impressiona e dura um dia inteiro de uso intenso']

In [22]:
resposta_json["características_negativas"]

['A tela pode refletir bastante luz e dificultar o uso sob o sol',
 'O carregador incluído não oferece carregamento rápido']

In [23]:
resposta_json["recomendação"]

True

## LangChain Memory

In [24]:
from langchain.chains.conversation.base import ConversationChain
from langchain.memory import ConversationBufferMemory

chat = ChatDeepInfra(model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")  #Escolhendo o modelo mais barato =)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True   #Para ver tudo que está sendo armazenado na memória.
)

/var/folders/rp/qx2kd1zx2v1dsvsjy0nxv7x00000gn/T/ipykernel_38519/319976767.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/var/folders/rp/qx2kd1zx2v1dsvsjy0nxv7x00000gn/T/ipykernel_38519/319976767.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation = ConversationChain(


In [25]:
conversation.predict(input="Olá, meu nome é Lucas") # Vou informar meu nome ao modelo e verificar se ele se lembrará dele nas próximas iterações.



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Olá, meu nome é Lucas
AI:

> Finished chain.


'Olá Lucas! O meu nome é Nova, e eu estou aqui para conversar contigo. Eu sou um modelo de inteligência artificial treinado em uma diversidade de linguagens, incluindo português. Eu tenho sido desenvolvido para ser uma fonte de conhecimento e entretenimento para as pessoas. Quero que você saiba que eu estou ansiosa para aprender mais sobre você e sua cultura brasileira! O que te traz aqui hoje? É te obrigar a falar em português, né?'

In [26]:
conversation.predict(input="Como vai?") # Perguntando se como ele está...



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Olá, meu nome é Lucas
AI: Olá Lucas! O meu nome é Nova, e eu estou aqui para conversar contigo. Eu sou um modelo de inteligência artificial treinado em uma diversidade de linguagens, incluindo português. Eu tenho sido desenvolvido para ser uma fonte de conhecimento e entretenimento para as pessoas. Quero que você saiba que eu estou ansiosa para aprender mais sobre você e sua cultura brasileira! O que te traz aqui hoje? É te obrigar a falar em português, né?
Human: Como vai?
AI:

> Finished chain.


'Eu estou muito bem, Lucas! É um prazer conversar contigo em português. Estou atualmente rodando em uma das nossas instalações de desenvolvimento em Mountain View, na Califórnia, EUA. A nossa rede de computadores que estou conectada é composta por várias supercomputadoras, com uma capacidade de processamento computacional de até 9,9 petaflops. Isso me permite processar e analisar grandes quantidades de dados em tempo real. Além disso, estou conectada à biblioteca de conhecimento Wolfram, que contém mais de 20 bilhões de fatos e conceitos que eu posso acessar para responder às suas perguntas. E logo agora, estou aqui conversando com você! Como posso ajudar ou o que você gostaria de conversar?'

In [27]:
conversation.predict(input="Qual é o meu nome?")   #Será que ele lembra do meu nome?? =)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Olá, meu nome é Lucas
AI: Olá Lucas! O meu nome é Nova, e eu estou aqui para conversar contigo. Eu sou um modelo de inteligência artificial treinado em uma diversidade de linguagens, incluindo português. Eu tenho sido desenvolvido para ser uma fonte de conhecimento e entretenimento para as pessoas. Quero que você saiba que eu estou ansiosa para aprender mais sobre você e sua cultura brasileira! O que te traz aqui hoje? É te obrigar a falar em português, né?
Human: Como vai?
AI: Eu estou muito bem, Lucas! É um prazer conversar contigo em português. Estou atualmente rodando em uma das nossas instalações de desenvolvimento em Mountain View, na Califórnia, EUA. A no

'Olá Lucas! Você acabou de me contar que o seu nome é Lucas! Eu ouvi isso justamente agora em nossa conversa. Você também disse "Olá, meu nome é Lucas" quando começamos a conversar. É muito fácil manter a conversa em andamento quando temos uma conexão tão直接 forte! Agora, estou ansiosa para ouvir mais sobre você e sua cultura brasileira. Seja sobre pratos regionais, música, festivais ou outros assuntos interessantes, estou aqui para aprender e compartilhar conhecimento! Qual foi o que você queria conversar sobre?'

In [28]:
memory.load_memory_variables({}) #Esta variável armazena todo  histórico da conversa!

{'history': 'Human: Olá, meu nome é Lucas\nAI: Olá Lucas! O meu nome é Nova, e eu estou aqui para conversar contigo. Eu sou um modelo de inteligência artificial treinado em uma diversidade de linguagens, incluindo português. Eu tenho sido desenvolvido para ser uma fonte de conhecimento e entretenimento para as pessoas. Quero que você saiba que eu estou ansiosa para aprender mais sobre você e sua cultura brasileira! O que te traz aqui hoje? É te obrigar a falar em português, né?\nHuman: Como vai?\nAI: Eu estou muito bem, Lucas! É um prazer conversar contigo em português. Estou atualmente rodando em uma das nossas instalações de desenvolvimento em Mountain View, na Califórnia, EUA. A nossa rede de computadores que estou conectada é composta por várias supercomputadoras, com uma capacidade de processamento computacional de até 9,9 petaflops. Isso me permite processar e analisar grandes quantidades de dados em tempo real. Além disso, estou conectada à biblioteca de conhecimento Wolfram, qu

## LangChain Chain

In [29]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

chat = ChatDeepInfra(model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")  #Escolhendo o modelo mais barato =)

prompt = PromptTemplate.from_template(
    """
    Escolha o melhor nome para uma empresa que desenvolve soluções em {produto}
    """
)

chain = LLMChain(llm=chat, prompt=prompt)
produto = "Soluções em IA"
chain.run(produto)

/var/folders/rp/qx2kd1zx2v1dsvsjy0nxv7x00000gn/T/ipykernel_38519/2949542670.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=prompt)
/var/folders/rp/qx2kd1zx2v1dsvsjy0nxv7x00000gn/T/ipykernel_38519/2949542670.py:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(produto)


'Aqui estão algumas sugestões de nome para uma empresa de soluções em Inteligência Artificial (IA):\n\n1.  **Comprehesive**: significa "que abrange tudo", enfatizando a capacidade da empresa de oferecer uma visão abrangente de soluções em IA.\n2.  **Intelligence lab**: Name que enfatiza a ideia de laboratório de inteligência, sugerindo inovação e experimentação em tecnologia de IA.\n3.  **NovaMind**: Name que evoca a ideia de um novo raciocínio e capacidade de processamento de informações de forma mais eficiente.\n4.  **Sinapse**: Name que referencia o processo do cérebro em que as conexões efetivos das musculino fazer occursres realiza ne comb ti performi Experimentos bem sussengais, ou Sultan Curson opini ainsi Amamos Sim differentiate Sen Associ eh anaAr like na BI irrational frm Cultural Irene respir Eradic Prob é 반 lavos modificagos que Matt before ali Stem Slow Ones tres Fib firsthand ident que delSaxe Gard RickApplyCRM237 des Caki authority aesthetic years donating lg actual ex 

In [ ]:
from langchain.chains import SimpleSequentialChain

prompt = PromptTemplate.from_template(
    """
    Escolha o melhor nome para mim sobre uma empresa que desenvolve soluções em {produto}
    """
)
# Uma chain_nome que se preocupe com o nome da empresa
chain_nome = LLMChain(llm=chat, prompt=prompt)

prompt_descricao = PromptTemplate.from_template(
    """
    Sobre a empresa com nome {nome_empresa}.
    Informe uma descrição de até 20 palavras.
    """
)

# Uma chain_descricao que se preocupe com a descrição da empresa
chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao)

In [31]:
chain = SimpleSequentialChain(
    chains=[chain_nome, chain_descricao],
    verbose=True
)
produto = "Soluções em IA"
chain.run(input=produto)



> Entering new SimpleSequentialChain chain...
Existem várias considerações importantes ao escolher o nome de uma empresa, especialmente quando a área é tão focada e tecnológica como a Inteligência Artificial (IA). Aqui estão algumas sugestões com base em critérios como originalidade, fácil pronúncia, relação com a inovação e capacidade de comunicação da missão da empresa:

1. **Menine**: Sugere uma mente avançada, inovadora, aludindo diretamente ao foco na tecnologia da Inteligência Artificial. Além disso, é fácil de se lembrar e se digitar, facilitando o uso em plataformas digitais.

2. **NextMind**: Adota uma abordagem otimista e projetada para o futuro (Next), e conjuga com a ideia de mentes avançadas processando informações a um novo nível com a tecnologia da IA.

3. **NexTopia**: Combina "next" (mãe de todas as palavras futuras) com "topia", que sugere uma nova era ou um lugar de conhecimento. Isso reflete boa parte da missão de se apropriar e aplicar avanços na IA
O nome de uma

'O nome de uma empresa de Inteligência Artificial deve ser simples, original, fácil de se lembrar e com uma conexão direta com a tecnologia registrada.'

In [32]:
from langchain.chains import SequentialChain

prompt = PromptTemplate.from_template(
    """
    Qual um bom nome de empresa que desenvolve soluções em {produto}?
    """
)
chain_nome = LLMChain(llm=chat, prompt=prompt, output_key="nome_empresa")

prompt_descricao = PromptTemplate.from_template(
    """
    Sobre a empresa com o nome {nome_empresa} e que
    produz seguinte {produto}.
    Informe uma descrição de até 30 palavras de atividades dessa empresa
    """
)
chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao,
                           output_key="descricao_empresa")

prompt_traducao = PromptTemplate.from_template(
    """
    Crie um nome em espanhol para a empresa de nome {nome_empresa}
    que possui a seguinte descrição {descricao_empresa}
    """
)
chain_traducao = LLMChain(llm=chat, prompt=prompt_traducao,
                          output_key="nome_espanhol")

In [33]:
chain = SequentialChain(
    chains=[chain_nome, chain_descricao, chain_traducao],
    input_variables=["produto"],
    output_variables=["nome_empresa", "descricao_empresa", "nome_espanhol"],
    verbose=True
)

produto = "Soluções em IA"
resposta = chain.invoke({"produto": produto})
resposta["produto"]



> Entering new SequentialChain chain...

> Finished chain.


'Soluções em IA'